In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '/gpfs/fs2/scratch/awhite38_lab/cfdsr/alcfd')
import utilities as utils

Defining input parameters

In [3]:
inlet_D = np.array([200]) # [um]
expansion_angle = np.array([40]) # [degrees]
inlet_v = np.array([0.5]) # [m/s]
inputs = [inlet_D, expansion_angle, inlet_v]

In [5]:
# Using blood as fluid
rho = 1060 # [kg m^-3]
muo = 0.004 # [Pa s]
re_number = utils.check_laminar(inlet_D*1e-6, inlet_v, rho, muo)
print(re_number)

(array([26.5]), array([], dtype=int64))


In [17]:
import os
import shutil
def run_model_expansion(inlet_D, expansion_angle, inlet_v, rho=1060, muo=0.004, debug=False, run_parallel=False, name=None):
    if name is None:
        name = ''
    path = os.getcwd()
    inputs_dir = path + '/inputs' + f'_{name}.txt'
    outputs_dir = path + '/outputs' + f'_{name}.txt'
    input_types = (list, int, float, np.float64, np.int)
    inputs = [inlet_D, expansion_angle, inlet_v]
    for i, input in enumerate(inputs):
        if isinstance(input, input_types):
            inputs[i] = np.array(input)[np.newaxis,...]
    inlet_D, expansion_angle, inlet_v = inputs
    input_dict = {'D': inlet_D, 'Angle': expansion_angle,
                  'V_in': inlet_v, 'outputs_file': outputs_dir}
    df = pd.DataFrame(input_dict)
    re_number, index_turbulent = utils.check_laminar(inlet_D*1e-6, inlet_v, rho, muo)
    if index_turbulent.size != 0:
        for i in index_turbulent:
            df = df.drop(index=i)
        print(
            f'Warning: Dropped inputs {index_turbulent} as they result a turbulent flow.')
    if not df.empty:
        print('Inputs to the model are:\n{}'.format(df))
        df.to_csv(inputs_dir, header=True, sep='\t', index=None)
        print(f'Model inputs are saved in:\n {inputs_dir}') 
        # re_number, a = check_laminar(pipe_D, rho, inlet_v, muo)
        wb = '/scratch/dfoster_lab/ansys2020R2/v202/Framework/bin/Linux64/runwb2'
        script = '/scratch/awhite38_lab/cfdsr/alcfd/archived_models/expansion/script.wbjn'
        if not run_parallel:
            shutil.copyfile(
                inputs_dir, '/scratch/awhite38_lab/cfdsr/alcfd/archived_models/expansion/inputs.txt')
        else:
            ProjectPath = f'/scratch/awhite38_lab/cfdsr/alcfd/archived_models/expansion/expansion_{name}.wbpj'
            input_file = inputs_dir
            with open(script, "r") as f:
                lines = f.readlines()
                for m, line in enumerate(lines):
                    if line.startswith('input_file'):
                        lines[m] = f'input_file = "{input_file}"\n'
                    if line.startswith('    ProjectPath'):
                        lines[m] = f'    ProjectPath="{ProjectPath}",\n'
            script_parallel = f'/scratch/awhite38_lab/cfdsr/alcfd/archived_models/expansion/script_parallel_{name}.wbjn'
            with open(script_parallel, "w") as f:
                lines = "".join(lines)
                f.write(lines)
            script = script_parallel
        if debug:
            cmdline = f'{wb} -B -R {script} > run_log_{name}.txt'
        else:
            cmdline = f'{wb} -B -R {script}'
        try:
                os.system(cmdline)
        except Exception:
                print('Failed to launch ANSYS Workbench!')
                sys.exit(0)
        outputs = pd.read_csv(outputs_dir, delimiter='\t')
        outputs = outputs.drop(columns=['Mesh max size', 'Inlet R', 'inlet_L_by_inlet_D', 'expansion_L_by_inlet_D'])
        print(f'Results are stored in:\n {outputs_dir}.')
        if run_parallel:
            shutil.rmtree(
                f'/scratch/awhite38_lab/cfdsr/alcfd/archived_models/expansion/expansion_{name}_files')
            os.remove(
                f'/scratch/awhite38_lab/cfdsr/alcfd/archived_models/expansion/expansion_{name}.wbpj')
            os.remove(script)
        return outputs
    else: 
        print('Model cannot run. All inputs resulted a turbulent flow.')


In [19]:
outputs = run_model_expansion(*inputs, name='test', debug=True)
outputs

<ipython-input-17-98b4e0f79caf>:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  input_types = (list, int, float, np.float64, np.int)


Inputs to the model are:
     D  Angle  V_in                                       outputs_file
0  200     40   0.5  /gpfs/fs2/scratch/awhite38_lab/cfdsr/alcfd/exa...
Model inputs are saved in:
 /gpfs/fs2/scratch/awhite38_lab/cfdsr/alcfd/examples/inputs_test.txt
Results are stored in:
 /gpfs/fs2/scratch/awhite38_lab/cfdsr/alcfd/examples/outputs_test.txt.


,Inlet D,Expansion angle,Mesh size,Viscosity,Density,Inlet P,Inlet V,System vol,Backflow percent,Backflow vol
0,200.0,40.0,0.000025,0.004,1060.0,15998.7,0.5,3.528010e-10,48.895581,1.725041e-10


In [ ]:
# /scratch/dfoster_lab/ansys2020R2/v202/Framework/bin/Linux64/runwb2 -B -R /scratch/awhite38_lab/cfdsr/alcfd/archived_models/expansion/script.wbjn